# Instacart Data Exploration 

__Problem Description:__
Whether you shop from meticulously planned grocery lists or let whimsy guide your grazing, our unique food rituals define who we are. Instacart, a grocery ordering and delivery app, aims to make it easy to fill your refrigerator and pantry with your personal favorites and staples when you need them. After selecting products through the Instacart app, personal shoppers review your order and do the in-store shopping and delivery for you.

Instacart’s data science team plays a big part in providing this delightful shopping experience. Currently they use transactional data to develop models that predict which products a user will buy again, try for the first time, or add to their cart next during a session. Recently, Instacart open sourced this data - see their blog post on 3 Million Instacart Orders, Open Sourced.

In this competition, Instacart is challenging the Kaggle community to use this anonymized data on customer orders over time __to predict which previously purchased products will be in a user’s next order.__ They’re not only looking for the best model, Instacart’s also looking for machine learning engineers to grow their team.

__File descriptions:__

Each entity (customer, product, order, aisle, etc.) has an associated unique id. Most of the files and variable names should be self-explanatory.

* __aisles.csv__

 aisle_id,aisle  
 1,prepared soups salads  
 2,specialty cheeses  
 3,energy granola bars  
 ...

* __departments.csv__

 department_id,department  
 1,frozen  
 2,other  
 3,bakery  
 ...

* __order\_products_*.csv__

 These files specify which products were purchased in each order. order_products_prior.csv contains previous order contents for all customers. 'reordered' indicates that the customer has a previous order that contains the product. Note that some orders will have no reordered items. You may predict an explicit 'None' value for orders with no reordered items. See the evaluation page for full details.

 order_id,product_id,add_to_cart_order,reordered  
 1,49302,1,1  
 1,11109,2,1  
 1,10246,3,0  
 ... 

* __orders.csv__

 This file tells to which set (prior, train, test) an order belongs. You are predicting reordered items only for the test set orders. 'order_dow' is the day of week.

 order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order  
 2539329,1,prior,1,2,08,  
 2398795,1,prior,2,3,07,15.0  
 473747,1,prior,3,3,12,21.0  
 ...

* __products.csv__

 product_id,product_name,aisle_id,department_id
 1,Chocolate Sandwich Cookies,61,19  
 2,All-Seasons Salt,104,13  
 3,Robust Golden Unsweetened Oolong Tea,94,7  
 ...

* __sample\_submission.csv__

 order_id,products
 17,39276  
 34,39276  
 137,39276  
 ...

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import cross_val_score

from sklearn import (
    tree,
    metrics,
    linear_model,
    ensemble,
    neighbors)

from sklearn.neighbors import KNeighborsClassifier
import re
import os
import gc

%matplotlib inline

## Initializing Data Directories and Data Filenames.

### Location of the dataset, which will be used to build the dataframes

In [2]:
ddir = 'Instacart Data'
print os.path.abspath(ddir)

/Users/NewAdmin/Documents/Anwar/DS Class/DS Final Project/Instacart Data


### List of the files to be used

In [3]:
aisles = ('aisles.csv')
depts = ('departments.csv')
ordprodprior = ('order_products__prior.csv')
ordprodtrain = ('order_products__train.csv')
orders = ('orders.csv')
products = ('products.csv')
sample_submission = ('sample_submission.csv')

## Function: buildDataFrame()

In [4]:
#-------------------------------------
# buildDataFrame()
#-------------------------------------
def buildDataFrame(sFilename = ""):
    print 'Loading ', sFilename
    df = pd.read_csv(os.path.join(ddir, sFilename))
    print 'Total Rows:', df.shape[0]
    return df

### Building individual Dataframes

In [5]:
df_aisles = buildDataFrame(aisles)
df_depts = buildDataFrame(depts)
df_ord = buildDataFrame(orders)
df_opp = buildDataFrame(ordprodprior)
df_opt = buildDataFrame(ordprodtrain)
df_prod = buildDataFrame(products)
print 'Done.'

Loading  aisles.csv
Total Rows: 134
Loading  departments.csv
Total Rows: 21
Loading  orders.csv
Total Rows: 3421083
Loading  order_products__prior.csv
Total Rows: 32434489
Loading  order_products__train.csv
Total Rows: 1384617
Loading  products.csv
Total Rows: 49688
Done.


### fixing datatypes for the dataframe

In [6]:
df_prod.aisle_id = df_prod.aisle_id.astype('category')
df_prod.department_id = df_prod.department_id.astype('category')
df_ord.eval_set = df_ord.eval_set.astype('category')

### Preparing Train and Test from orders

In [7]:
df_ord_prior = df_ord[df_ord.eval_set == 'prior']
df_ord_train = df_ord[df_ord.eval_set == 'train']
df_ord_test = df_ord[df_ord.eval_set == 'test']
df_ord_prior.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [8]:
df_ord_train.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
10,1187899,1,train,11,4,8,14.0
25,1492625,2,train,15,1,11,30.0
49,2196797,5,train,5,0,11,6.0
74,525192,7,train,21,2,11,6.0
78,880375,8,train,4,1,14,10.0


In [9]:
df_ord_test.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0


In [10]:
list_user_id = df_ord_train.user_id.unique().tolist()

total_user_id_prior = df_ord_prior["user_id"].nunique() 
total_user_id_exist_train_but_not_in_prior = df_ord_prior[~df_ord_prior.user_id.isin(list_user_id)]["user_id"].nunique()
print "Total User Count in Prior:{}\nTotal User exists in Train but NOT in Prior:{}\nDifference:{}".format(total_user_id_prior, 
                                                                                                           total_user_id_exist_train_but_not_in_prior, 
                                                                                                           total_user_id_prior - total_user_id_exist_train_but_not_in_prior)

print ("\n")

list_user_id = df_ord_test.user_id.unique().tolist()

total_user_id_test = df_ord_test["user_id"].nunique() 
total_user_id_exist_test_but_not_in_prior = df_ord_prior[~df_ord_prior.user_id.isin(list_user_id)]["user_id"].nunique()
print "Total User Count in Prior:{}\nTotal User exists in Test but NOT in Prior:{}\nDifference:{}".format(total_user_id_prior, 
                                                                                                           total_user_id_exist_test_but_not_in_prior, 
                                                                                                           total_user_id_prior - total_user_id_exist_test_but_not_in_prior)


Total User Count in Prior:206209
Total User exists in Train but NOT in Prior:75000
Difference:131209


Total User Count in Prior:206209
Total User exists in Test but NOT in Prior:131209
Difference:75000


### Merging orders + prior merging orders + train

In [11]:
df_opp = pd.merge(df_opp, df_ord_prior, on='order_id')
df_opt = pd.merge(df_opt, df_ord_train, on='order_id')

In [12]:

def removeUnwantedColumns(df):
    colist = df.columns.tolist()
    for col in colist:
        if col.endswith("_x") or col.endswith("_y"):
            df.drop(col, axis=1, inplace=True)

removeUnwantedColumns(df_opp)
removeUnwantedColumns(df_opt)


In [13]:
df_opp.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0
1,2,28985,2,1,202279,prior,3,5,9,8.0
2,2,9327,3,0,202279,prior,3,5,9,8.0
3,2,45918,4,1,202279,prior,3,5,9,8.0
4,2,30035,5,0,202279,prior,3,5,9,8.0


In [14]:
df_opt.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,112108,train,4,4,10,9.0
1,1,11109,2,1,112108,train,4,4,10,9.0
2,1,10246,3,0,112108,train,4,4,10,9.0
3,1,49683,4,0,112108,train,4,4,10,9.0
4,1,43633,5,1,112108,train,4,4,10,9.0


## Calculating Product stats

In [15]:
df_prod.reset_index(inplace=True)

df_temp = df_opp.groupby('product_id').agg({'order_id':'nunique', 
                                            'reordered':'sum'}).reset_index()

df_temp['reordered_rate'] = (df_temp.reordered.astype(float)/df_temp.order_id).astype(float)

df_temp.columns = ['product_id', 
                   'p_total_orders', 
                   'p_sum_reordered', 
                   'p_reordered_rate']

df_temp.reset_index(inplace=True)

df_prod = pd.merge(df_temp,df_prod, on='product_id').reset_index()

#dropping unwanted columns
removeUnwantedColumns(df_prod)

del df_temp
df_prod.head()

,index,product_id,p_total_orders,p_sum_reordered,p_reordered_rate,product_name,aisle_id,department_id
0,0,1,1852,1136,0.613391,Chocolate Sandwich Cookies,61,19
1,1,2,90,12,0.133333,All-Seasons Salt,104,13
2,2,3,277,203,0.732852,Robust Golden Unsweetened Oolong Tea,94,7
3,3,4,329,147,0.446809,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,4,5,15,9,0.600000,Green Chile Anytime Sauce,5,13


## Calculating User Stats

In [16]:
df_temp = df_opp.groupby('user_id').agg({'days_since_prior_order':'mean', 
                                         'order_id':'nunique'}).reset_index()

df_temp.columns = ['user_id', 
                   'u_total_orders', 
                   'u_avg_days_since_prior_order']
#df_temp.head()

# calculating user stats 2
df_temp1 = df_opp.groupby('user_id').agg({'product_id':'nunique'}).reset_index()
df_temp1['all_products'] = df_opp.groupby('user_id')['product_id'].apply(set)

#print df_temp1['all_products'].isnull().sum()
#print df_temp1['all_products'].dtype

df_temp1['all_products'].fillna('', inplace=True)
df_temp1['total_distinct_items'] = df_temp1['all_products'].map(len)

df_temp1.columns = ['user_id', 
                    'u_total_products', 
                    'u_all_products', 
                    'u_total_distinct_items']


df_users = pd.merge(df_temp, df_temp1, on='user_id').reset_index()

df_users['u_avg_cart'] = (df_users.u_total_products / df_users.u_total_orders).astype(float)

removeUnwantedColumns(df_users)

df_users.set_index("user_id", inplace=True, drop=False)
df_users.drop("index", axis=1, inplace=True)

del df_temp
del df_temp1

df_users.head()

,user_id,u_total_orders,u_avg_days_since_prior_order,u_total_products,u_all_products,u_total_distinct_items,u_avg_cart
user_id,,,,,,,
1,1,10,20.259259,18,,0,1.800000
2,2,14,15.967033,102,"{17122, 196, 26405, 13032, 39657, 12427, 25133...",18,7.285714
3,3,12,11.487179,33,"{45066, 2573, 18961, 23, 32792, 22559, 13351, ...",102,2.750000
4,4,5,15.357143,17,"{17668, 39190, 44683, 21903, 14992, 21137, 324...",33,3.400000
5,5,4,14.500000,23,"{26576, 21573, 17769, 25623, 35469, 37646, 366...",17,5.750000


### Estabilishing new key that denotes user_id+product_id

In [17]:
upSeed = 100000 # seed for number of product_id digits to reverse calculate.

# upSeed Test Code
#ll = [[33120,202279, 415570208], [28985,202279, 415566073], [9327,202279, 415546415], [45918,202279, 415583006], [30035,202279, 415567123]]
#for l in ll: print (l[0] + l[1] * 100000)


df_opp['user_product_id'] = (df_opp.product_id + df_opp.user_id * upSeed)
df_opt['user_product_id'] = (df_opt.product_id + df_opt.user_id * upSeed)
df_opp.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_product_id
0,2,33120,1,1,202279,prior,3,5,9,8.0,20227933120
1,2,28985,2,1,202279,prior,3,5,9,8.0,20227928985
2,2,9327,3,0,202279,prior,3,5,9,8.0,20227909327
3,2,45918,4,1,202279,prior,3,5,9,8.0,20227945918
4,2,30035,5,0,202279,prior,3,5,9,8.0,20227930035


In [18]:
df_opt.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_product_id
0,1,49302,1,1,112108,train,4,4,10,9.0,11210849302
1,1,11109,2,1,112108,train,4,4,10,9.0,11210811109
2,1,10246,3,0,112108,train,4,4,10,9.0,11210810246
3,1,49683,4,0,112108,train,4,4,10,9.0,11210849683
4,1,43633,5,1,112108,train,4,4,10,9.0,11210843633


In [19]:
df_temp_grped = df_opp.groupby('user_product_id')
df_temp = pd.DataFrame()
df_temp['up_max_order_id'] = df_temp_grped.order_id.max() 
df_temp['up_max_order_number'] = df_temp_grped.order_number.max() 
df_temp['up_total_orders'] = df_temp_grped.order_id.nunique()
df_temp['up_max_add_to_cart_order'] = df_temp_grped.add_to_cart_order.sum()

df_temp.reset_index(inplace=True)
df_user_product = df_temp
del df_temp

df_user_product['product_id'] = (df_user_product['user_product_id'] % upSeed).astype(int)
df_user_product['user_id'] = (df_user_product['user_product_id'] / upSeed).astype(int)

df_user_product.head()

,user_product_id,up_max_order_id,up_max_order_number,up_total_orders,up_max_add_to_cart_order,product_id,user_id
0,100196,3367565,10,10,14,196,1
1,110258,3367565,10,9,30,10258,1
2,110326,431534,5,1,5,10326,1
3,112427,3367565,10,10,33,12427,1
4,113032,2550362,10,3,19,13032,1


#### Validation

In [20]:
df_user_product[df_user_product.user_product_id == 14126415872]

,user_product_id,up_max_order_id,up_max_order_number,up_total_orders,up_max_add_to_cart_order,product_id,user_id
9128446,14126415872,2141752,20,5,63,15872,141264


In [21]:
max(df_opp[ (df_opp.user_id == 141264) & (df_opp.product_id == 15872)]['order_id'] )

2141752

In [22]:
max(df_opp[ df_opp.user_id == 141264]['order_number'])

20

In [23]:
df_opt.shape[0]

In [59]:
df_opt.set_index(['order_id', 'product_id'], inplace=True, drop=False)

In [60]:
df_opt.head()

order_id  product_id  add_to_cart_order  reordered  \
order_id product_id                                                       
1        49302              1       49302                  1          1   
         11109              1       11109                  2          1   
         10246              1       10246                  3          0   
         49683              1       49683                  4          0   
         43633              1       43633                  5          1   

                     user_id eval_set  order_number  order_dow  \
order_id product_id                                              
1        49302        112108    train             4          4   
         11109        112108    train             4          4   
         10246        112108    train             4          4   
         49683        112108    train             4          4   
         43633        112108    train             4          4   

                     order_hour_of_day  days_since_prior_order  \
order_id product_id                                              
1        49302                      10                     9.0   
         11109                      10                     9.0   
         10246                      10                     9.0   
         49683                      10                     9.0   
         43633                      10                     9.0   

                     user_product_id  
order_id product_id                   
1        49302           11210849302  
         11109           11210811109  
         10246           11210810246  
         49683           11210849683  
         43633           11210843633

In [92]:
df_ord_train.columns.tolist()

['order_id',
 'user_id',
 'eval_set',
 'order_number',
 'order_dow',
 'order_hour_of_day',
 'days_since_prior_order']

## Build Features

In [94]:
###--------------------------------
### THIS FUNCTION RUNS FOREVER
###--------------------------------

def buildFeatures(df, buildTarget=False, useFullDataset=False):
    i = 0
    v_list_of_products = []
    v_list_of_orders = []
    v_list_of_users = []
    v_list_of_targets = []
    
    print "Total Rows to be processed:{}".format(df.shape[0])
    
    for dfrow in df.itertuples():
        if i%10000 == 0: 
            print "Processed:",i
            
        v_order_id = dfrow.order_id
        v_user_id = dfrow.user_id
        #print("({}-{})".format(v_order_id, v_user_id)),
        
        try:

            #print i, v_order_id, v_user_id
            v_all_products_user_bought = df_users.u_all_products[v_user_id]
            #print v_all_products_user_bought

            #expanding the prodcuts from the set we built in the users df.
            v_list_of_products += v_all_products_user_bought
            v_list_of_orders += [v_order_id] * len(v_all_products_user_bought) # to make it as same size as v_list_of_products
            v_list_of_users += [v_user_id] * len(v_all_products_user_bought)
            
            if buildTarget:
                #for product in v_all_products_user_bought:
                v_list_of_targets += [(v_order_id, product) in df_opt.index for product in v_all_products_user_bought]
                

            if not useFullDataset:
                if i > 10000: break
            i += 1
            
        except KeyError as e:
            #print "ValueError({0}): {1}".format(e. , e.strerror)
            print "OrderID: {} UserID: {}".format(v_order_id, v_user_id)
            continue
     
    #build the master features dataframe
    df_master = pd.DataFrame({"order_id":v_list_of_orders, "product_id":v_list_of_products, "user_id":v_list_of_users})
    
    if buildTarget:
        df_master['target'] = v_list_of_targets
    
    df_master = pd.DataFrame.merge(df_master, df_prod, on=['product_id'])
    df_master = pd.DataFrame.merge(df_master, df_users, on='user_id')
    df_master = pd.DataFrame.merge(df_master, df_user_product, on=['user_id','product_id'])
    
    
    return df_master

In [95]:
df_features = buildFeatures(df_ord_train, buildTarget=True, useFullDataset=True)
df_features = pd.merge(df_features, df_ord_train, on=['order_id', 'user_id'])

removeUnwantedColumns(df_features)

Total Rows to be processed:131209
Processed: 0
Processed: 10000


In [96]:
print "Total Rows (Orders Train):", df_features.shape[0]

df_features[df_features.target == True]

20292
['order_id', 'product_id', 'user_id', 'target', 'index', 'p_total_orders', 'p_sum_reordered', 'p_reordered_rate', 'product_name', 'aisle_id', 'department_id', 'u_total_orders', 'u_avg_days_since_prior_order', 'u_total_products', 'u_all_products', 'u_total_distinct_items', 'u_avg_cart', 'user_product_id', 'up_max_order_id', 'up_max_order_number', 'up_total_orders', 'up_max_add_to_cart_order', 'eval_set', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']
<bound method DataFrame.info of        order_id  product_id  user_id  target  index  p_total_orders  \
0       1492625       13176        2   False  13172          379450   
1       1492625       41787        2    True  41779           35413   
2       1162736       17122      261   False  17118           13880   
3       3174585       24852      328   False  24848          472565   
4       3174585       33401      328   False  33395           13450   
5        451330       25146      558   False  25142  

,order_id,product_id,user_id,target,index,p_total_orders,p_sum_reordered,p_reordered_rate,product_name,aisle_id,...,user_product_id,up_max_order_id,up_max_order_number,up_total_orders,up_max_add_to_cart_order,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1,1492625,41787,2,True,41779,35413,23015,0.649903,Bartlett Pears,24,...,241787,3194192,7,2,32,train,15,1,11,30.0
11,2354554,24852,779,True,24848,472565,398609,0.843501,Banana,24,...,77924852,2924540,5,1,9,train,7,0,8,19.0
15,2073686,24852,973,True,24848,472565,398609,0.843501,Banana,24,...,97324852,2635064,25,5,51,train,26,0,11,8.0
17,599773,13176,1143,True,13172,379450,315913,0.832555,Bag of Organic Bananas,24,...,114313176,3026885,3,2,10,train,4,6,18,14.0
18,599773,21903,1143,True,21899,241921,186884,0.772500,Organic Baby Spinach,123,...,114321903,3026885,3,2,3,train,4,6,18,14.0
25,2721390,49235,1681,True,49224,76360,59672,0.781456,Organic Half & Half,53,...,168149235,3281805,7,3,29,train,9,2,9,5.0
30,1979738,43352,1884,True,43344,57640,39507,0.685409,Raspberries,32,...,188443352,2893230,17,6,27,train,22,1,17,9.0
31,3001884,13176,2143,True,13172,379450,315913,0.832555,Bag of Organic Bananas,24,...,214313176,2904817,16,4,26,train,20,0,16,12.0
33,1074427,19348,2477,True,19344,29345,22559,0.768751,Fat Free Milk,84,...,247719348,1855819,3,1,9,train,6,4,9,30.0
38,2606288,48205,2872,True,48194,16829,9176,0.545249,Spinach,123,...,287248205,886889,1,1,17,train,6,3,13,30.0


## Functions: Model

In [ ]:
def printCrossValidation (cls, X,y, model_name):
    roc_auc = cross_val_score(cls, X, y, scoring='roc_auc', cv=5).mean()
    print '\n',model_name,':'
    print ' Score:', cls.score(X, y)
    print ' ROC AUC:', roc_auc
    print ' Accuracy:', cross_val_score(cls, X, y, scoring='accuracy', cv=5).mean()
    print ' Recall:', cross_val_score(cls, X, y, scoring='recall', cv=5).mean()
    return roc_auc
    

def fitRandomForest(X, y):
    cls = ensemble.RandomForestClassifier(n_estimators=500,
                             criterion='gini',
                            max_features='auto',
                            max_depth = 8,
                            min_samples_split = 2,
                            min_samples_leaf = 20,
                            bootstrap = True) 
    cls.fit(X, y)
    
    cols =  X.columns
    feature_importances = cls.feature_importances_
    features_df = pd.DataFrame({'features': features, 'importance': feature_importances})
    features_df.sort_values('importance', inplace=True, ascending=False)
    
    printCrossValidation (cls, X,y, "Random Forest")
    print ' Feature importance: ',features_df.head(25)
    return cls

def fitGradiantBoostingClassifier(X, y):
    cls = ensemble.GradientBoostingClassifier(loss='deviance', 
                                              learning_rate=0.1, 
                                              n_estimators=500, 
                                              subsample=1.0, 
                                              criterion='friedman_mse', 
                                              min_samples_split=2, 
                                              min_samples_leaf=20, 
                                              min_weight_fraction_leaf=0.0, 
                                              max_depth=8, 
                                              min_impurity_decrease=0.0, 
                                              min_impurity_split=None, 
                                              init=None, 
                                              random_state=None, 
                                              max_features=None, 
                                              verbose=0, 
                                              max_leaf_nodes=None, 
                                              warm_start=False, 
                                              presort='auto') 
    cls.fit(X, y)
    
    cols =  X.columns
    feature_importances = cls.feature_importances_
    features_df = pd.DataFrame({'features': features, 'importance': feature_importances})
    features_df.sort_values('importance', inplace=True, ascending=False)
    
    printCrossValidation (cls, X,y, "Grandiant Boosting")
    print ' Feature importance: ',features_df.head(25)
    return cls


In [97]:
features = ['p_total_orders'
            , 'p_sum_reordered'
            , 'p_reordered_rate'
            , 'aisle_id'
            , 'department_id'
            , 'u_total_orders'
            , 'u_avg_days_since_prior_order'
            , 'u_total_products'
            , 'u_total_distinct_items'
            , 'u_avg_cart'
            , 'up_max_order_id'
            , 'up_max_order_number'
            , 'up_total_orders'
            , 'up_max_add_to_cart_order'
            , 'order_number'
            , 'order_dow'
            , 'order_hour_of_day'
            , 'days_since_prior_order']

target = 'target'

X = df_features[features]
y = df_features[target]
df_features[features]

,p_total_orders,p_sum_reordered,p_reordered_rate,aisle_id,department_id,u_total_orders,u_avg_days_since_prior_order,u_total_products,u_total_distinct_items,u_avg_cart,up_max_order_id,up_max_order_number,up_total_orders,up_max_add_to_cart_order,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,379450,315913,0.832555,24,4,14,15.967033,102,18,7.285714,2168274,1,1,12,15,1,11,30.0
1,35413,23015,0.649903,24,4,14,15.967033,102,18,7.285714,3194192,7,2,32,15,1,11,30.0
2,13880,9377,0.675576,24,4,8,3.647059,49,31,6.125000,2116059,7,1,8,9,3,17,4.0
3,472565,398609,0.843501,24,4,12,15.666667,63,243,5.250000,2272583,9,3,4,13,2,11,27.0
4,13450,7390,0.549442,21,16,12,15.666667,63,243,5.250000,451857,7,1,10,13,2,11,27.0
5,24918,15803,0.634200,31,7,27,8.105960,167,48,6.185185,1450877,26,4,56,28,5,12,1.0
6,24950,14300,0.573146,116,1,27,8.105960,167,48,6.185185,1532626,18,1,3,28,5,12,1.0
7,17332,10343,0.596757,96,20,27,8.105960,167,48,6.185185,970433,21,2,13,28,5,12,1.0
8,379450,315913,0.832555,24,4,9,7.107143,65,23,7.222222,2713750,9,1,24,10,1,12,7.0
9,13494,9759,0.723210,115,7,4,27.812500,42,51,10.500000,3353185,4,1,11,5,1,16,7.0


In [100]:
clsRF = fitRandomForest(X,y)


Random Forest :
 Score: 0.857677902622
 ROC AUC: 0.782852199102
 Accuracy: 0.847476747281
 Recall: 0.100052151239
 Feature importance:                          features  importance
12               up_total_orders    0.239729
13      up_max_add_to_cart_order    0.099077
2               p_reordered_rate    0.087991
5                 u_total_orders    0.077319
14                  order_number    0.073636
1                p_sum_reordered    0.065028
7               u_total_products    0.056231
10               up_max_order_id    0.054799
0                 p_total_orders    0.048392
11           up_max_order_number    0.043404
6   u_avg_days_since_prior_order    0.039818
9                     u_avg_cart    0.035166
17        days_since_prior_order    0.021642
8         u_total_distinct_items    0.019877
16             order_hour_of_day    0.014283
15                     order_dow    0.011923
3                       aisle_id    0.008493
4                  department_id    0.003192


In [101]:
clsGBT = fitGradiantBoostingClassifier(X,y)


Grandiant Boosting :
 Score: 0.997289572245
 ROC AUC: 0.754068356894
 Accuracy: 0.840725429577
 Recall: 0.22290055707
 Feature importance:                          features  importance
10               up_max_order_id    0.123060
13      up_max_add_to_cart_order    0.108933
6   u_avg_days_since_prior_order    0.096258
9                     u_avg_cart    0.078599
2               p_reordered_rate    0.076827
8         u_total_distinct_items    0.070864
7               u_total_products    0.061259
12               up_total_orders    0.050924
0                 p_total_orders    0.049493
1                p_sum_reordered    0.047270
11           up_max_order_number    0.046568
17        days_since_prior_order    0.042338
16             order_hour_of_day    0.039318
3                       aisle_id    0.029388
15                     order_dow    0.028233
5                 u_total_orders    0.022189
14                  order_number    0.021393
4                  department_id    0.007087


In [129]:
df_features_test = buildFeatures(df_ord_test, buildTarget = False, useFullDataset= True)
df_features_test.head()

Total Rows to be processed:75000
Processed: 0
Processed: 10000
Processed: 20000
Processed: 30000
Processed: 40000
Processed: 50000
Processed: 60000
Processed: 70000


,order_id,product_id,user_id,index,p_total_orders,p_sum_reordered,p_reordered_rate,product_name,aisle_id,department_id,...,u_avg_days_since_prior_order,u_total_products,u_all_products,u_total_distinct_items,u_avg_cart,user_product_id,up_max_order_id,up_max_order_number,up_total_orders,up_max_add_to_cart_order
0,2774568,47766,3,47755,176815,134044,0.758103,Organic Avocado,24,4,...,11.487179,33,"{45066, 2573, 18961, 23, 32792, 22559, 13351, ...",102,2.750000,347766,3225766,12,9,34
1,2774568,16797,3,16793,142951,99802,0.698155,Strawberries,24,4,...,11.487179,33,"{45066, 2573, 18961, 23, 32792, 22559, 13351, ...",102,2.750000,316797,3160850,9,3,12
2,1192143,47209,28,47198,213584,170131,0.796553,Organic Hass Avocado,24,4,...,12.296089,106,"{44548, 32263, 45066, 16398, 43352, 44051, 491...",220,4.416667,2847209,2195945,23,1,5
3,1192143,24852,28,24848,472565,398609,0.843501,Banana,24,4,...,12.296089,106,"{44548, 32263, 45066, 16398, 43352, 44051, 491...",220,4.416667,2824852,2760669,24,6,19
4,1192143,21137,28,21133,264683,205845,0.777704,Organic Strawberries,24,4,...,12.296089,106,"{44548, 32263, 45066, 16398, 43352, 44051, 491...",220,4.416667,2821137,2695156,8,1,2


In [130]:
df_features_test = pd.merge(df_features_test, df_ord_test, on=['order_id', 'user_id'])
removeUnwantedColumns(df_features_test)

print "Total Rows (Orders Test):", df_features_test.shape[0]
#print df_features_test.columns.tolist()

X = df_features_test[features]
df_features_test["predicted"] = clsRF.predict(X)
df_features_test.head()



156055
['order_id', 'product_id', 'user_id', 'index', 'p_total_orders', 'p_sum_reordered', 'p_reordered_rate', 'product_name', 'aisle_id', 'department_id', 'u_total_orders', 'u_avg_days_since_prior_order', 'u_total_products', 'u_all_products', 'u_total_distinct_items', 'u_avg_cart', 'user_product_id', 'up_max_order_id', 'up_max_order_number', 'up_total_orders', 'up_max_add_to_cart_order', 'eval_set', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']


,order_id,product_id,user_id,index,p_total_orders,p_sum_reordered,p_reordered_rate,product_name,aisle_id,department_id,...,up_max_order_id,up_max_order_number,up_total_orders,up_max_add_to_cart_order,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,predicted
0,2774568,47766,3,47755,176815,134044,0.758103,Organic Avocado,24,4,...,3225766,12,9,34,test,13,5,15,11.0,True
1,2774568,16797,3,16793,142951,99802,0.698155,Strawberries,24,4,...,3160850,9,3,12,test,13,5,15,11.0,False
2,1192143,47209,28,47198,213584,170131,0.796553,Organic Hass Avocado,24,4,...,2195945,23,1,5,test,25,1,16,8.0,False
3,1192143,24852,28,24848,472565,398609,0.843501,Banana,24,4,...,2760669,24,6,19,test,25,1,16,8.0,False
4,1192143,21137,28,21133,264683,205845,0.777704,Organic Strawberries,24,4,...,2695156,8,1,2,test,25,1,16,8.0,False


In [171]:
def convertSetToFlatString(sets):
    s = ""
    for item in sets:
        s += str(item) + " " 
    return s

In [177]:
df = df_features_test[df_features_test.predicted == 1]
#['user_id','order_id', 'product_id', 'predicted']
df = pd.DataFrame(df_features_test.groupby(["user_id", "order_id"]).product_id.apply(set))
df["product_id"] = df["product_id"].apply(convertSetToFlatString)
df.head()

,,product_id
user_id,order_id,
3,2774568,16797 47766
11,1376945,42736 26209 17794 43352
12,1356845,13176 42736 17794
20,1980631,9387
22,139655,35221
